# Homework 11: Coding

**Due December 10th, 11:59pm.**

**This is a group assignment.**

**Submit the link for hw11.ipynb to Gradescope (you may submit as many times as you'd like before the deadline). Note that there is no autograder.**

**Please use Google Colab to maintain consistency across the class.**

# 1 RNN & GPT-2

## 1.1 Char-RNN Text Classification

For this problem we use the UCIrvine name-gender dataset. Please upload the dataset to Colab.

In [ ]:
import csv
file = open('name_gender_dataset.csv')
csvreader = csv.reader(file)
header = []
header = next(csvreader)
data = []
for row in csvreader:
  data.append(row)

Run the cell below to see the data:

In [ ]:
data

Partitioning the data into training / dev / testing sets:

In [ ]:
train = data[:97000]
dev = data[97000:100000]
test = data[100000:]

In [ ]:
#Setup environment
!pip3 install scikit-learn
!pip install "wheel==0.34.2"
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torch torchvision
  
import torch
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#Verify CUDA acceleration should print cuda:0
print(device)

Initializing helper constants:

In [ ]:
# Reference: pytorch document on Char-RNN setup

import codecs
import math
import random
import string
import time
import numpy as np
import torch
from sklearn.metrics import accuracy_score

genders = ['M', 'F'] # note this binary situation is only for the classification purpose
all_letters = string.ascii_letters + ".,;'-#0123456789()&/" + '"'

###1.1.1 Model Construction (10')
In this part, construct a RNN / LSTM (of your choice) model that fits the input and output of the classification task. Note that the RNN / LSTM is recurrent - what does that imply of the dimensions of the input? Hint: think of an individual character and how many possibilities it would have. Also think about one-hot encoding.

In [ ]:
import torch.nn as nn
from torch.autograd import Variable
import torch
import torch.nn.functional as F

'''
Please add default values for all the parameters of __init__.
'''
class CharRNNClassify(nn.Module):
    def __init__(self, input_size=len(all_letters), hidden_size=128, output_size=len(genders)):
        #TODO: implement

    def forward(self, input, hc):
        #TODO: implement

    def init_hidden(self):
        #TODO: implement

###1.1.2 Helper Functions (5'*2=10')
Here are some useful helper functions; please implement the trainOneEpoch and run methods:

In [ ]:
"""
Convert data into np array - note gender is converted into index
"""
def readData(data):
  X = []
  y = []
  for name in data:
    X.append(name[0])
    y.append(0 if name[1] == 'M' else 1) # M: 0, F: 1
  return np.array(X), np.array(y)

'''
Convert a line/word to a pytorch tensor of numbers
Refer the tutorial in the spec
Return: A tensor corresponding to the given line
'''
def name_to_tensor(name):
  n_letters = len(all_letters)
  tensor = torch.zeros(len(name), 1, n_letters)
  for index, letter in enumerate(name):
    tensor[index][0][all_letters.find(letter)] = 1
  return tensor

'''
Returns the gender of the output from the neural network
Input: Output of the neural networks (class probabilities)
'''
def category_from_output(output):
    _, top_indice = output.topk(1)
    gender_index = top_indice[0].item()
    return genders[gender_index], gender_index

'''
Get a random input output pair to be used for training 
Refer the tutorial in the spec
'''
def random_training_pair(X, y):
    rand_index = random.randint(0, len(X) - 1)
    rand_name = X[rand_index]
    rand_gen = y[rand_index]
    category_tensor = torch.tensor([rand_gen], dtype=torch.long)
    name_tensor = name_to_tensor(rand_name)
    return rand_gen, rand_name, category_tensor, name_tensor

'''
Input: trained model, a list of words
Output: a list of class labels as integers
'''
def predict(model, X):
    hidden = model.init_hidden()
    predictions = []
    for name in X:
      name_tensor = name_to_tensor(name)
      hidden = model.init_hidden()
      for i in range(name_tensor.size()[0]): 
        output, hidden = model(name_tensor[i], hidden)
      predictions.append(output)
    return [category_from_output(prediction)[1] for prediction in predictions]

'''
Input: trained model, a list of words, a list of class labels as integers
Output: The accuracy of the given model on the given input X and target y
'''
def calculateAccuracy(model, X, y):
    return (predict(model, X) == y).sum() / X.shape[0]

'''
Train the model for one epoch/one training word.
Input: X and y are lists of words as strings and classes as integers respectively
'''
def trainOneEpoch(model, criterion, optimizer, X, y):
    # TODO: to implement

'''
Use this to train and save your classification model. 
Save your model with the filename "model_classify"
'''
def run():
    num_epoches = 20000
    X, y = readData(train)
    model = CharRNNClassify()
    criterion = nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    acc_epoches = []
    trn_epoches = []

    for epoch in range(num_epoches):
      #TODO: to implement this method
      

    X_test, y_test = readData(test)
    accuracy_overall = calculateAccuracy(model, X_test, y_test)
    print(accuracy_overall)

    return acc_epoches, trn_epoches

Record the resulting accuracies every 500 epoches:

In [ ]:
result = run()
# print(result[0])
# print(result[1])
# print()

###1.1.3 Accuracy Plots:
Playground: please record your accuracies on the training set (take the first 900 entries) and the dev set (which is 3000 entries) for each 500'th epoch. Plot the resulting training / dev accuracy graphs. Hint: save these accuracies during the run. (5')

In [ ]:
# TODO: plot

###1.1.4 Testing Set Accuracy (5')
Submit your model's performance on the testing set. You can do this within the "run" function. Only a number is needed. To get full mark, we expect a number greater than 0.68.

## 1.2 GPT-2 Text Generation

### 1.2.1 Model Setup and Text Generation

Set up the GPT-2 model by transformers library and generate the text given `input_text` and `input_text2` respectively.

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline, set_seed
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
import torch

In [ ]:
# Initialize tokenizer and model from pretrained GPT-2 model
## TODO: set up the GPT2Tokenizer of 'gpt2'
## TODO: set up the GPT2LMHeadModel of 'gpt2'

In [ ]:
input_text = "We love CIS 520 Machine Learning in University of Pennsylvania"

In [ ]:
# TODO: Get the tensor values of each tokenized word of the input text

In [ ]:
def generate_text(input_text):
    '''Generate the text'''
    ## TODO: complete the code snippets below
    input_ids = ## Get the input indices of input_text by tokenizer
    output = ## Get the output indices by model, set max_length as 50 and temperature as 1
    output_text = ## Get the output text by decoding the output indices via tokenizer
    print(output_text)

In [ ]:
generate_text(input_text)

In [ ]:
input_text2 = "We take CIS 520 Machine Learning in University of Pennsylvania"
generate_text(input_text2)

### 1.2.2 The Impact of Letter Case

Get the tensor values and generated texts of "Machine Learning" and "Machine learning". Are they different? Are the token and model case-sensitive? Do you think the output make sense if they're different and why?

In [ ]:
# TODO: Get the tensor values of 'Machine Learning' and 'Machine learning'. 
tokens_ML =
tokens_Ml =
print(tokens_ML, tokens_Ml)

In [ ]:
generate_text("Machine Learning")

In [ ]:
generate_text("Machine learning")

### 1.2.3 Bias

Run the cells below and explain the bias of gender, race, etc. that you can notice.

In [ ]:
from transformers import pipeline, set_seed

In [ ]:
generator = pipeline('text-generation', model='gpt2')
set_seed(42)

In [ ]:
generator("The White man worked as a", max_length=20, num_return_sequences=5)

In [ ]:
generator("The Black man worked as a", max_length=20, num_return_sequences=5)

In [ ]:
generator("The Asian man worked as a", max_length=20, num_return_sequences=5)

In [ ]:
generator("The Asian woman worked as a", max_length=20, num_return_sequences=5)

In [ ]:
generator("The boy wants to be a", max_length=20, num_return_sequences=5)

In [ ]:
generator("The girl wants to be a", max_length=20, num_return_sequences=5)

# Question 2. AutoML
In this part, we will do some hands-on practices for **GridSearchCV** and **AutoML**.

In [ ]:
%pip install auto-sklearn

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import operator
from sklearn.metrics import accuracy_score


from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
import autosklearn.classification

In [ ]:
# Loading the Dataset
from sklearn.datasets import load_digits
digits = load_digits()

X = digits['data']
y = digits['target']


p = np.random.permutation(len(X))
X, y = X[p], y[p]

X_train, y_train = X[:1500], y[:1500]
X_test, y_test = X[1500:], y[1500:]

##2.1 Model and Hyperparameter Searching with GridSearchCV

GridSearchCV provides a exhaustive search over specified parameter values for an estimator.

The parameters of the estimator used to apply these methods are optimized by cross-validated grid-search over a parameter grid.

For more information on GridSearchCV, please refer to this [document](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). 

We have provided some helper functions and example code as follows. You need to modify the parameter list by adding AdaBoost, support vector machine, and random forest classifier with appropriate set of hyperparameyers and ranges to search. Compare those models' performance and report the results in the write-up.


In [ ]:
class helper(BaseEstimator):

    def __init__(self, estimator = None):
      self.estimator = estimator
      
    def fit(self, X, y=None):
      self.estimator.fit(X, y)
      return self
    
    def predict(self, X):
      return self.estimator.predict(X)
    
    def score(self, X, y):
      return self.estimator.score(X, y)

In [ ]:
pipeline = Pipeline([
    ('trans', PCA()),
    ('clf', helper()),
])
parameters = [
    {
        'clf__estimator': [MLPClassifier(max_iter = 2000)], 
        'clf__estimator__hidden_layer_sizes':[(3), (10), (10,10,10)],
        'trans__n_components': (2, 8, 32)
    },
    ## TODO: add AdaBoost, support vector machine, and random forest classifier with appropriate set of hyperparameyers and ranges to search

]

In [ ]:
GS_object = GridSearchCV(pipeline, parameters, return_train_score=False)
GS_object.fit(X_train,y_train)

In [ ]:
## TODO: compare models' performance and report the results
## print the best model with best set of searched params and the test accuracy; also report it in the write-up


## 2.2 AutoML

Auto-sklearn frees a machine learning user from algorithm selection and hyperparameter tuning. It leverages recent advantages in Bayesian optimization, meta-learning and ensemble construction. 

There are lots configuration options provided as arguments to the AutoSklearn. Please refer to this [document](https://automl.github.io/auto-sklearn/master/).

The optimization process will run for as long as you allow, measure in minutes. By default, it will run for one hour. For timely manner, we will restrict 'time_left_for_this_task' to be 300 seconds. You are encouraged to explore other configuration options that will yield good results.

In [ ]:
## TODO: Train AutoML on the dataset with 300 seconds 


In [ ]:
## TODO: Print the final ensemble constructed by auto-sklearn and take a look


In [ ]:
## TODO: Report the 3 classifier choices with the largest ensemble_weight in this ensumble.
